<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Capter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# ライブラリインストール
# 各ライブラリの働きはREADME参照
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0
import torch
from transformers import BertJapaneseTokenizer, BertModel


# トークナイザの定義
# トークナイザ:文章をトークン分割して、BERTに入力可能なようにする
# 東北大学が作った事前学習ずみBERTモデルをまず用意
# それを用いて学習済みトークナイザをロード（BertJapaneseTokenizerはおそらくライブラリtransformersに含まれている）
# (このモデルはMeCabを用いて単語分割し、WordPieceを用いてその単語をトークンに分割する。wordPieceの語彙はtokenizer.vocabからアクセス)
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)


# まずは文章をトークン化してみよう
tokenizer.tokenize('明日は自然言語処理の勉強をしよう')




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う']

In [ ]:
# トークナイザの定義
# トークナイザ:文章をトークン分割して、BERTに入力可能なようにする
# 東北大学が作った事前学習ずみBERTモデルをまず用意
# それを用いて学習済みトークナイザをロード（BertJapaneseTokenizerはおそらくライブラリtransformersに含まれている）
# (このモデルはMeCabを用いて単語分割し、WordPieceを用いてその単語をトークンに分割する。wordPieceの語彙はtokenizer.vocabからアクセス)
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

In [7]:
# まずは文章をトークン化してみよう
tokenizer.tokenize('明日は自然言語処理の勉強をしよう')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う']

In [8]:
# マシンラーニングなどの複合語は、先頭ではないサブワードには＃＃がつく
# サブワード分割はWordPieceのお仕事
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う']

In [9]:
# 未知語を表すトークン：[UNK]
tokenizer.tokenize('機械学習を中国語にすると机器学习という')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'という']

In [13]:
# トークン化が完了したので、次は得られたトークンをトークンIDに変換しよう（BERTには数値で入力）
# この作業を符号化(encode)と呼ぶ

# encode()は、tokenize()の結果をIDに変換したものの先頭に[CLS] (文章の先頭のトークン)、最後に[SEP] (文章終わりのトークン)をおく


input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう')
print(tokenizer.tokenize('明日は自然言語処理の勉強をしよう'))
print(input_ids)

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う']
[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 3]


In [14]:
# 逆変換（ID→トークン）はconvert_ids_to_tokensで行う
tokenizer.convert_ids_to_tokens(input_ids)

['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '[SEP]']

In [18]:
# BERTでは通常、複数の文章をまとめて処理する -> 文章のトークン列の長さ（系列長）を揃えないとダメ
# 文章が短ければ、特殊トークン[PAD]を追加する
# 文章が長ければ、末尾のトークンではみ出したものは削除


# 例えばこれを符号化すると、以下のようになる
text = "明日の天気は晴れだ。"

# textを符号化している
# 系列長は12(特殊トークン含む)であり、足りなければ[PAD], 多すぎれば切り捨て(truncation)
encoding = tokenizer(
    text, max_length=12, padding='max_length', truncation=True
)


# input_ids : トークンの符号化結果
# token_type_ids' ： 2つの文章を同時に入力した場合、どちらの文章に属しているのかを判断する
# attention_mask : [PAD]の位置で0, それ以外で1. PADはattentionでは考慮しなくて良い
print('# encoding : ')
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding['input_ids']))



# encoding : 
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [20]:
# max_lengthを短くすると、文章が削られる

# textを符号化している
# 系列長は6(特殊トークン含む)であり、足りなければ[PAD], 多すぎれば切り捨て(truncation)
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=True
)


# input_ids : トークンの符号化結果
# token_type_ids' ： 2つの文章を同時に入力した場合、どちらの文章に属しているのかを判断する
# attention_mask : [PAD]の位置で0, それ以外で1. PADはattentionでは考慮しなくて良い
print('# encoding : ')
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding['input_ids']))

# encoding : 
{'input_ids': [2, 11475, 5, 11385, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


In [23]:
# 複数文章をまとめて符号化

text_list = ['明日の天気は晴れだ。', 'パソコンが急に動かなくなった。']

# 系列長は10(特殊トークン含む)であり、足りなければ[PAD], 多すぎれば切り捨て(truncation)
encoding = tokenizer(
    text_list, max_length=10, padding='max_length', truncation=True
)


# input_ids : トークンの符号化結果
# token_type_ids' ： 2つの文章を同時に入力した場合、どちらの文章に属しているのかを判断する
# attention_mask : [PAD]の位置で0, それ以外で1. PADはattentionでは考慮しなくて良い
print('# encoding : ')
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][1]))

# encoding : 
{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]']
['[CLS]', 'パソコン', 'が', '急', 'に', '動か', 'なく', 'なっ', 'た', '[SEP]']


In [24]:
# tokenizerのpaddingを「longest」に設定すると
# 最長の文章の長さに系列長を合わせる

encoding = tokenizer(
    text_list, padding='longest'
)

print('# encoding : ')
print(encoding)
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][1]))


# encoding : 
{'input_ids': [[2, 11475, 5, 11385, 9, 16577, 75, 8, 3, 0, 0], [2, 6311, 14, 1132, 7, 16084, 332, 58, 10, 8, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '。', '[SEP]', '[PAD]', '[PAD]']
['[CLS]', 'パソコン', 'が', '急', 'に', '動か', 'なく', 'なっ', 'た', '。', '[SEP]']


In [26]:
# transformers(ライブラリ)の提供するBERTに値を入力する際には、データ型をtorch.Tensorにしておく必要がある
# tokenizerの引数に「return_tensors='pt'」と付け加えることでデータ型変更可能

encoding = tokenizer(
    text_list, padding='longest', return_tensors='pt'
)

print(encoding)

{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,     8,     3,     0,
             0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     8,
             3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
